<a href="https://colab.research.google.com/github/hokoro/Crawling/blob/main/publicapihomework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os # 파일 관리 관련 모듈
import sys # 변수 함수 제어 관련 모듈
import urllib.request#url 열기 관련 모듈
import datetime # 일정 관련 모듈
import time #시간 관련 모듈
import json#json 파일 관련 모듈
import pandas as pd  #pandas DataFrame module

In [ ]:
#내가 받은 서비스 키 
ServiceKey = "ql8%2F7V%2FHyASEWj4DXINa9eC6rPjAiH9fDI6KVm61Gp0AZVYW1HyftwuT%2B3ZrvrOZUekDWRfbZtUqgtxwKU8%2BnQ%3D%3D"

In [ ]:
def getRequestUrl(url):
      req = urllib.request.Request(url) #URL 을 보낼 객체를 생성
      try:
          response = urllib.request.urlopen(req) #요청객체에서 응답 받은 데이터를 response 객체에 저장
          if response.getcode() == 200: #코드가 200 이면 요청한 처리를 정상 처리로 성공
              print("[%s] Url Request Success" % datetime.datetime.now()) # 성공 처리 시간을 출력
              return response.read().decode('utf-8') #파이썬 셀창에 출력하고 응답을 utf-8 형식으로 디코딩 하여 반환
      except Exception as e: #요청이 처리되지 않을 경우
          print(e) #에러 메시지를 출력
          print("[%s] Error for URL : %s" % (datetime.datetime.now(), url)) #실패한 시간과 , url 을 출력
          return None  #성공하지 못하였기 때문에 반환값은 None 으로 아무것도 반환하지 않음


#변수 설명

## 매개변수
### yyyymm: 수집한 연월
### nat_cd : 수집 대상 국가의 코드 
### ed_cd : 수집할 데이터 종류 

## 지역변수
### service_url : 앤드 포인트 주소
### parameters : url 에 추가할 매개변수
### url : service_url 과 parameters 를 연결하여 완성한 url
### responseDecode : getRequestUrl 을 호출하여 반환받은 응답 객체 

## 메서드
### getRequestUrl :url 요청에 대한 응답 데이터를 받음
### json.loads() : json 형식으로 받은 응답 데이터인 responseDecode 를 파이썬 객체로 읽음 

In [ ]:
def getTourismStatsItem(yyyymm,national_code , ed_cd):
  #우리가 사용할 url 을 년도 , 국가 코드 , 구분 을 넣어서 하나의 url 로 만드는 과정
  service_url = 'http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList' 

  parameters = "?_type=json&serviceKey="+ServiceKey
  parameters += "&YM=" +yyyymm
  parameters +="&NAT_CD="+national_code
  parameters +="&ED_CD="+ed_cd

  url = service_url + parameters
  print(url)
  #url 이 완성되었으면 해당 url 의 응답 데이터를 받기 위해 getRequestUrl 을 호출 하여 응답 데이터 를 저장
  responseDecode = getRequestUrl(url)
 
 
  if (responseDecode == None): #응답데이터를 못받은 경우
    return None
  else: #응답 데이터를 받은 경우
    return json.loads(responseDecode) #받은 데이터를 파이썬 객체를 읽음 





In [ ]:
def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):																										# getTourismStatsService 함수 정의
  jsonResult = []																																																		# json으로 저장할 리스트
  result = []																																																				# csv로 저장할 리스트
  natName = ''																																																			# 국가 이름 변수
  dataEND = "{0}{1:0>2}".format(str(nEndYear), str(12))																															# 마지막 데이터의 연월 변수
  isDataEnd = 0																																																			# 데이터의 끝을 확인할 변수
  for year in range(nStartYear, nEndYear+1):																																				# 시작연도 부터 마지막연도+1까지 반복문
    for month in range(1, 13):																																											# 1,2,3,...10,11,12월까지 반복
      if (isDataEnd == 1): break																																										# 데이터가 끝이변 break
      yyyymm = "{0}{1:0>2}".format(str(year), str(month))																														# 연도와 월을 맞추어 저장
      jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd)																													# jsonData에 월 데이터를 저장
      if (jsonData['response']['header']['resultMsg'] == 'OK'):																											# 저장한 데이터가 ok인지 확인
        if jsonData['response']['body']['items'] == '':          																										# 데이터가 더 없는 경우
          isDataEnd = 1																																															# isDataEnd에 1값을 추가해 끝을 알림
          dataEND = "{0}{1:0>2}".format(str(year), str(month-1))																										# dataEND에 마지막 데이터 날짜-1을 저장
          print("데이터 없음... \n 제공되는 통계 데이터는 %s년 %s월까지입니다."%(str(year), str(month-1)))					# 마지막 데이터 연월을 출력
          break																																																			# break
        print(json.dumps(jsonData, indent = 4, sort_keys = True, ensure_ascii = False))															# jsonData를 출력
        natName = jsonData['response']['body']['items']['item']['natKorNm']		
																		# 'natKorNm'의 값을 저장
        natName = natName.replace(' ', '')																																					# 띄어쓰기를 제거
        num = jsonData['response']['body']['items']['item']['num']																									# 'num' : 월 데이터 수
        ed = jsonData['response']['body']['items']['item']['ed']																										# 'ed' : 출입국 구분
        print('[%s_%s : %s ]'%(natName, yyyymm, num))																																# 국가명, 연월, 데이터 수 출력
        print('--------------------------------------------------')																									# 구분선 출력
        jsonResult.append({'nat_name': natName, 'nat_cd': nat_cd, 'yyyymm': yyyymm, 'visit_cnt': num})							# 국가명, 국가코드, 날짜, 데이터 수를 jsonResult에 저장
        result.append([natName, nat_cd, yyyymm, num])																																# 국가명, 국가코드, 날짜, 데이터 수를 result에 저장
  return (jsonResult, result, natName, ed, dataEND)			
  

In [ ]:
def main():
  jsonResult = [] 
  result = []
  print("<<국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
  nat_cd = input('국가 코드를 입력하세요 : ') #국가 코드 입력
  nStartYear = int(input('데이터를 몇 년 부터 수집할까요? :')) #수집 년도의 시작점 입력
  nEndYear = int(input('데이터를 몇 년 까지 수집할까요? :')) #수집년도의 끝점 입력

  ed_cd = "E" #E = 방한 외래 관광객
  jsonResult,result,natName,ed,dataEND = getTourismStatsService(nat_cd,ed_cd,nStartYear,nEndYear) #수집 함수를 호출하여 해당 결과를 순서대로 저장 
  #해당 국가 ,구분데이터 ,시작연도 등을 파일 이름으로 사용하여 json 형태로 저장 , 쓰기모드 , 인코딩 형식 = utf-8
  with open('./%s_%s_%d_%s.json'%(natName , ed , nStartYear , dataEND) , 'w' , encoding = 'utf8') as outfile:
    jsonFile = json.dumps(jsonResult , indent = 4 , sort_keys = True , ensure_ascii = False) #jsom 파일 파이썬 형식으로 가져오기 
    outfile.write(jsonFile) #json 파일 쓰기 

  
  columns = ['입국자국가','국가코드','입국연월','입국자 수'] #columns
  result_df = pd.DataFrame(result,colums = columns) #DataFrame 형식으로 저장 
  #데이터 프레임을 csv 파일 로 변환 하여 저장 
  result_df.to_csv('./%s_%s_%d_%s.json'%(natName , ed , nStartYear , dataEND) , index = False , encoding = 'cp949')


In [ ]:
if __name__ == '__main__':
  main() #main function call 

<<국내 입국한 외국인의 통계 데이터를 수집합니다. >>
국가 코드를 입력하세요 : 112
데이터를 몇 년 부터 수집할까요? :2016
데이터를 몇 년 까지 수집할까요? :2017
http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList?_type=json&serviceKey=ql8%2F7V%2FHyASEWj4DXINa9eC6rPjAiH9fDI6KVm61Gp0AZVYW1HyftwuT%2B3ZrvrOZUekDWRfbZtUqgtxwKU8%2BnQ%3D%3D&YM=201601&NAT_CD=112&ED_CD=E
[2022-04-06 07:30:16.166272] Url Request Success
{
    "response": {
        "body": {
            "items": {
                "item": {
                    "ed": "방한외래관광객",
                    "edCd": "E",
                    "natCd": 112,
                    "natKorNm": "중  국",
                    "num": 521981,
                    "rnum": 1,
                    "ym": 201601
                }
            },
            "numOfRows": 10,
            "pageNo": 1,
            "totalCount": 1
        },
        "header": {
            "resultCode": "0000",
            "resultMsg": "OK"
        }
    }
}
중  국
[중국_201601 : 521981 ]
-------------------

TypeError: ignored

 {
        "nat_cd": "112",
        "nat_name": "중국",
        "visit_cnt": 254930,
        "yyyymm": "201706"
    },